In [1]:
# pip install -U sentence_transformers seaborn plotly numpy pandas
# !pip install -U dask

In [2]:
DATA_DIR = 'ga-project/data/'
# DATA_DIR = '/Users/annie/blah/blah/my-project/data/'
# WRONG:  DATA_DIR = '/Users/annie/blah/blah/my-project/data'  # no trailing slash
!pwd
!ls
!ls ga-project/data

/home/marco/Documents/Reference/Course-Materials/annie
2019.csv      analysis.ipynb	dtale-eda.ipynb     SWEETVIZ_REPORT.html
2020.csv      Archive.zip	final-project	    temp_plot.html
2021.csv      D4-download.csv	ga-project
2022.csv      D4-download.xlsx	stor-lab2
all-data.csv  dataprofile.html	sweetviz-eda.ipynb


wikiart_cleaned.csv  wikiart_scraped.csv


In [55]:
pio.renderers.default='vscode'

In [3]:
# import dask.dataframe as dd
# import dask.array as da
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE

In [4]:
from sklearn.cluster import KMeans

In [5]:
import plotly.express as px
import plotly.io as pio

In [6]:
from bertopic import BERTopic

In [7]:
#!pip install --user -U nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

## Text Preprocessing

In [8]:
# wikiart = dd.read_csv(
#     DATA_DIR+'wikiart_cleaned.csv').set_index('Unnamed: 0')
# # Reset the index and set a new index with the desired name
# wikiart = wikiart.reset_index(drop=True)
# wikiart['index'] = wikiart.index
# wikiart = wikiart.set_index('index')

wikiart = pd.read_csv(
    DATA_DIR+'wikiart_cleaned.csv', index_col=0)

wikiart.shape

(124170, 5)

In [9]:
wikiart.head()

,Style,Artwork,Artist,Date,Link
0,Early-Dynastic,Narmer Palette,Ancient Egypt,-3050,https://uploads3.wikiart.org/00265/images/anci...
1,Early-Dynastic,Box Inlay with a Geometric Pattern,Ancient Egypt,-3000,https://uploads2.wikiart.org/00244/images/anci...
2,Old-Kingdom,Khafre Enthroned,Ancient Egypt,-2570,https://uploads2.wikiart.org/00305/images/anci...
3,Middle-Kingdom,Stele of the Serpent King (Stela of Djet),Ancient Egypt,-3000,https://uploads7.wikiart.org/00305/images/anci...
4,Middle-Kingdom,"Laden Donkeys and Ploughing, Tomb of Djar",Ancient Egypt,-2035,https://uploads8.wikiart.org/00244/images/anci...


### Remove stop words in multiple languages

In [23]:
stops = set(stopwords.words('english'))
stops.update(stopwords.words('german'))
stops.update(stopwords.words('french'))
stops.update(stopwords.words('italian'))
stops.update(stopwords.words('spanish'))
stops.update(stopwords.words('greek'))
stops.update(stopwords.words('romanian'))
stops.update(stopwords.words('chinese'))
stops.update(stopwords.words('russian'))
stops.update(['the','The','THE'])
def remove_stop_words(text):
    text = text.split()
    text = [w for w in text if not w in stops]
    if not text:  # If the list is empty after removing stop words
        return None
    text = " ".join(text)
    return text

### Using Sentence Transformers to Calculate Embeddings

In [24]:
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [38]:
wikiart_sample = wikiart.sample(20000)
wikiart_sample.reset_index(inplace=True)
wikiart_sample['Artwork'] = wikiart_sample['Artwork'].apply(remove_stop_words)
wikiart_sample = wikiart_sample[wikiart_sample['Artwork'].notna()]
wikiart_sample.reset_index(drop=True, inplace=True)

In [39]:
corpus = wikiart_sample['Artwork']
embeddings = model.encode(corpus)
embeddings.shape

(19999, 512)

In [40]:
# Truncate artwork names to create labels
corpuslabels = [elem.split()[0] for elem in corpus]
len(corpuslabels)

19999

In [42]:
# Let's visualize the similarities in a heatmap to test whether we can discover topics
# Define a function that creates a heatmap for sentence similarity
def plot_similarity(labels, sim_, rotation=0):
    sns.set(rc={'figure.figsize': (18, 18)}, font_scale=0.8)
    g = sns.heatmap(sim_,
                    xticklabels=labels, yticklabels=labels,
                    vmin=0, vmax=1, cmap="YlOrRd")
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Semantic Textual Similarity")
    return g

In [43]:
# sim_matrix = np.inner(embeddings, embeddings)
# plot_similarity(corpuslabels, sim_matrix, 500)
# sim_matrix.shape

### Dimensionality Reduction - TSNE

In [44]:
X_tsne = TSNE(
    n_components=2,
    n_jobs=60,
    # verbose=1,
    # perplexity=5,
    # n_iter=1000,
    # learning_rate=50,
    # init='random',
).fit_transform(embeddings)

In [45]:
X_tsne.shape, len(corpus)

((19999, 2), 19999)

### Clustering Using K Means

In [46]:
kmeans = KMeans(n_clusters = 4, init='k-means++', n_init=10, max_iter=500) # Initializing KMeans
kmeans = kmeans.fit(embeddings) # Fitting with inputs
labels = kmeans.predict(embeddings) # Predicting the clusters

In [47]:
source = pd.DataFrame(
    {
        'x': X_tsne[:, 0],
        'y': X_tsne[:, 1],
        'Artwork': corpus,
        'Topic': labels.astype(pd.Categorical)
    })
    
fig = px.scatter(
    source,
    x='x',
    y='y',
    color='Topic',
    hover_data=['Artwork', 'Topic'],
    title="Topics",
    labels={'x': 'not directly interpretable', 'y': 'not directly interpretable'},
    #color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(width=600, height=600)
fig.show()
#pio.renderers.default = 'vscode'

### Clustering Using DBSCAN

### Topic Modeling: BERTopic

In [48]:
topic_model = BERTopic(language = 'multilingual', top_n_words = 9, verbose=True, nr_topics=10)
topics, probabilities = topic_model.fit_transform(corpus)

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

2023-09-05 23:18:13,824 - BERTopic - Transformed documents to Embeddings
2023-09-05 23:18:29,651 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-09-05 23:18:31,048 - BERTopic - Clustered reduced embeddings
2023-09-05 23:18:31,446 - BERTopic - Reduced number of topics from 421 to 10


In [49]:
topic_model.visualize_barchart(top_n_topics = 10).show()

## Topic Trends throughout Time

### 3000 BC - 0 CE

In [50]:
wikiart_bc = wikiart[wikiart['Date'] < 0]
wikiart_bc.reset_index(inplace=True)
wikiart_bc['Artwork'] = wikiart_bc['Artwork'].apply(remove_stop_words)
wikiart_bc

,index,Style,Artwork,Artist,Date,Link
0,0,Early-Dynastic,Narmer Palette,Ancient Egypt,-3050,https://uploads3.wikiart.org/00265/images/anci...
1,1,Early-Dynastic,Box Inlay Geometric Pattern,Ancient Egypt,-3000,https://uploads2.wikiart.org/00244/images/anci...
2,2,Old-Kingdom,Khafre Enthroned,Ancient Egypt,-2570,https://uploads2.wikiart.org/00305/images/anci...
3,3,Middle-Kingdom,Stele Serpent King (Stela Djet),Ancient Egypt,-3000,https://uploads7.wikiart.org/00305/images/anci...
4,4,Middle-Kingdom,"Laden Donkeys Ploughing, Tomb Djar",Ancient Egypt,-2035,https://uploads8.wikiart.org/00244/images/anci...
...,...,...,...,...,...,...
341,426,Geometric,Attic Middle Geometric Amphora Kerameikos,Ancient Greek Pottery,-800,https://uploads5.wikiart.org/00238/images/anci...
342,427,Geometric,Attic Late Geometric Amphora. From Athens. By ...,Ancient Greek Pottery,-750,https://uploads4.wikiart.org/00238/images/anci...
343,428,Geometric,Hirschfeld Krater,Ancient Greek Pottery,-735,https://uploads3.wikiart.org/00238/images/anci...
344,429,Geometric,Attic Late Geometric Pyxis Four Horses Lid. Fr...,Ancient Greek Pottery,-735,https://uploads2.wikiart.org/00238/images/anci...


In [51]:
corpus_bc = wikiart_bc['Artwork']
topic_model = BERTopic(language = 'multilingual', top_n_words = 10, verbose=True)
topics, probabilities = topic_model.fit_transform(corpus_bc)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2023-09-05 23:18:35,914 - BERTopic - Transformed documents to Embeddings
2023-09-05 23:18:38,629 - BERTopic - Reduced dimensionality
2023-09-05 23:18:38,637 - BERTopic - Clustered reduced embeddings


In [57]:
topic_model.visualize_barchart(top_n_topics = 5).show()

### Medieval Period (0 - 1500CE)

In [53]:
wikiart_me = wikiart[(wikiart['Date'] >= 0) & (wikiart['Date'] < 1500)]
wikiart_me.reset_index(inplace=True)
wikiart_me['Artwork'] = wikiart_me['Artwork'].apply(remove_stop_words)
wikiart_me

,index,Style,Artwork,Artist,Date,Link
0,341,Hellenistic,Mummy Portrait Man,Fayum portrait,50,https://uploads8.wikiart.org/00210/images/fayu...
1,342,Hellenistic,Mummy Portrait Young Woman,Fayum portrait,50,https://uploads8.wikiart.org/00210/images/fayu...
2,343,Hellenistic,Mummy Portrait Eirene,Fayum portrait,50,https://uploads8.wikiart.org/00210/images/fayu...
3,344,Hellenistic,Fayum Mummy Portrait,Fayum portrait,53,https://uploads8.wikiart.org/00210/images/fayu...
4,345,Hellenistic,Panel Painting Woman Blue Mantle,Fayum portrait,68,https://uploads1.wikiart.org/00210/images/fayu...
...,...,...,...,...,...,...
5801,123846,Post-classic-(c.-),Tripod Bowl,Aztec Art,1450,https://uploads7.wikiart.org/00294/images/azte...
5802,123847,Post-classic-(c.-),Tripod Vessel Date Glyph,Aztec Art,1450,https://uploads5.wikiart.org/00294/images/azte...
5803,123848,Post-classic-(c.-),Serpent Head,Aztec Art,1410,https://uploads6.wikiart.org/00294/images/azte...
5804,123849,Post-classic-(c.-),Coatlicue ('Serpent Skirt'),Aztec Art,1410,https://uploads3.wikiart.org/00294/images/azte...


In [56]:
corpus_me = wikiart_me['Artwork']
topic_model = BERTopic(language = 'multilingual', top_n_words = 10, verbose=True)
topics, probabilities = topic_model.fit_transform(corpus_me)
topic_model.visualize_barchart(top_n_topics = 5).show()

Batches:   0%|          | 0/182 [00:00<?, ?it/s]

2023-09-05 23:22:34,651 - BERTopic - Transformed documents to Embeddings
2023-09-05 23:22:39,030 - BERTopic - Reduced dimensionality
2023-09-05 23:22:39,155 - BERTopic - Clustered reduced embeddings


### Early Modern Period (1500 - 1800)

In [58]:
wikiart_earlymo = wikiart[(wikiart['Date'] >= 1500) & (wikiart['Date'] < 1800)]
wikiart_earlymo.reset_index(inplace=True)
wikiart_earlymo['Artwork'] = wikiart_earlymo['Artwork'].apply(remove_stop_words)
wikiart_earlymo

,index,Style,Artwork,Artist,Date,Link
0,999,Late-Byzantine/Palaeologan-Renaissance-(c.-–),Annunciation,Orthodox Icons,1500,https://uploads0.wikiart.org/00332/images/orth...
1,1005,Late-Byzantine/Palaeologan-Renaissance-(c.-–),Saint Demetrius,Orthodox Icons,1525,https://uploads1.wikiart.org/00332/images/orth...
2,1006,Late-Byzantine/Palaeologan-Renaissance-(c.-–),John Baptist - Angel Desert,Orthodox Icons,1575,https://uploads1.wikiart.org/00332/images/orth...
3,2098,International-Gothic,Virgin Child,Hans Holbein the Elder,1500,https://uploads3.wikiart.org/00311/images/hans...
4,2113,International-Gothic,Exterior Wings Frankfurt Dominican Altarpiece,Hans Holbein the Elder,1501,https://uploads6.wikiart.org/00311/images/hans...
...,...,...,...,...,...,...
17403,123763,Post-classic-(c.-),Xiuhmolpilli (1 Death),Aztec Art,1500,https://uploads2.wikiart.org/00294/images/azte...
17404,123764,Post-classic-(c.-),Dead Warrior Brazier,Aztec Art,1500,https://uploads6.wikiart.org/00294/images/azte...
17405,123765,Post-classic-(c.-),Flea,Aztec Art,1500,https://uploads2.wikiart.org/00294/images/azte...
17406,123766,Post-classic-(c.-),Hunchback,Aztec Art,1500,https://uploads7.wikiart.org/00294/images/azte...


In [59]:
corpus_earlymo = wikiart_earlymo['Artwork']
topic_model = BERTopic(language = 'multilingual', top_n_words = 10, verbose=True)
topics, probabilities = topic_model.fit_transform(corpus_earlymo)
topic_model.visualize_barchart(top_n_topics = 5).show()

Batches:   0%|          | 0/544 [00:00<?, ?it/s]

2023-09-05 23:25:17,133 - BERTopic - Transformed documents to Embeddings
2023-09-05 23:25:22,941 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-09-05 23:25:24,137 - BERTopic - Clustered reduced embeddings


### Modern (1800 - Present)

In [70]:
wikiart_mo = wikiart[(wikiart['Date'] >= 1800) & (wikiart['Date'] <= 2023)]
wikiart_mo.reset_index(inplace=True)
wikiart_mo['Artwork'] = wikiart_mo['Artwork'].apply(remove_stop_words)
wikiart_mo_sample = wikiart_mo.sample(10000) # Sampling because large number artworks in modern era
wikiart_mo_sample = wikiart_mo_sample[wikiart_mo_sample['Artwork'].notna()]
wikiart_mo_sample.reset_index(drop=True, inplace=True)
wikiart_mo_sample

,index,Style,Artwork,Artist,Date,Link
0,22841,Neoclassicism,Part Series Chronicling Uniforms Napoleon's Gr...,Carle Vernet,1812,https://uploads4.wikiart.org/00289/images/carl...
1,32082,Biedermeier,"Portrait young lady blue dress, lowered eyes",Josef Kriehuber,1866,https://uploads1.wikiart.org/00326/images/jose...
2,24213,Academicism,Calling David,Alexander Ivanov,1850,https://uploads7.wikiart.org/images/alexander-...
3,115675,Neo-Pop-Art,Cats,Oleksandr Hnylyzkyj,2011,https://uploads6.wikiart.org/00155/images/olek...
4,55211,Impressionism,A Needlewoman,Pierre-Auguste Renoir,1875,https://uploads6.wikiart.org/images/pierre-aug...
...,...,...,...,...,...,...
9994,66008,Socialist-Realism,Youth,Tetyana Yablonska,1969,https://uploads5.wikiart.org/00124/images/tety...
9995,38048,Naïve-Art-(Primitivism),Costume Design,Mikhail Larionov,1915,https://uploads4.wikiart.org/00124/images/mikh...
9996,78416,Abstract-Expressionism,Dusk,William Baziotes,1958,https://uploads5.wikiart.org/images/william-ba...
9997,22806,Neoclassicism,Part Series Chronicling Uniforms Napoleon's Gr...,Carle Vernet,1812,https://uploads5.wikiart.org/00289/images/carl...


In [71]:
corpus_mo = wikiart_mo_sample['Artwork']
topic_model = BERTopic(language = 'multilingual', top_n_words = 10, verbose=True)
topics, probabilities = topic_model.fit_transform(corpus_mo)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2023-09-05 23:36:46,593 - BERTopic - Transformed documents to Embeddings
2023-09-05 23:36:52,597 - BERTopic - Reduced dimensionality
2023-09-05 23:36:52,826 - BERTopic - Clustered reduced embeddings


In [72]:
topic_model.visualize_barchart(top_n_topics = 5).show()